<a href="https://colab.research.google.com/github/realpolarbearking/stockScreener/blob/main/SCANNER__S%26P600_Universe__Momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas_datareader --upgrade

     |████████████████████████████████| 109 kB 5.1 MB/s 
  Attempting uninstall: pandas-datareader
    Found existing installation: pandas-datareader 0.9.0
    Uninstalling pandas-datareader-0.9.0:
      Successfully uninstalled pandas-datareader-0.9.0


In [ ]:
from pandas_datareader import data as pdr
from scipy.stats import linregress
import matplotlib.pyplot as plt
import bs4 as bs
import requests
import pandas as pd
import numpy as np
import datetime as dt
import time

In [ ]:
# Variables
start_date = dt.datetime.now() - dt.timedelta(days=365 * 2)
end_date = dt.datetime.now()
index_name = 'SPY'
index_df = pdr.get_data_yahoo(index_name, start_date, end_date)

In [ ]:
#get S&P600 tickers from web
resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_600_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

tickers = []

for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[1].text
        
        tickers.append(ticker)

tickers = list(map(lambda s: s.strip(), tickers))
tickerdf = pd.DataFrame(tickers,columns=['ticker'])
print(tickerdf)

    ticker
0      AAN
1     AAON
2      AAT
3     AAWW
4     ABCB
..     ...
596   XNCR
597   XPER
598   ZEUS
599   ZUMZ
600   ZYXI

[601 rows x 1 columns]


In [ ]:
# Momentum Calculating Method 
"""...............................................
Momentum(Volatility Adjusted)
  = Exponential_Regression_Slope * R-Squared 
..............................................."""

def mom(closes):
    returns = np.log(closes)
    x = np.arange(len(returns))
    slope, _, rvalue, _, _ = linregress(x, returns)
    return ((1 + slope) ** 252) * (rvalue ** 2) 

def ROC(df, n):  
    M = df['Adj Close'].diff(n - 1)  
    N = df['Adj Close'].shift(n - 1)  
    ROC = pd.Series(((M / N) * 100), name = 'ROC_' + str(n))   
    return ROC

def RS(df, df1, n):  
    rs = ROC(df, n) / ROC(df1, n)
    return rs

# Position Sizing Method
"""...............................................
size = (AccountValue * RiskFactor) / ATR(20)
def wwma(values, n):
    return values.ewm(alpha=1/n, adjust=False).mean()

def atrMom(df, atrLength):
    data = df.copy()
    high = data['High']
    low = data['Low']
    close = data['Adj Close']
    data['tr0'] = abs(np.log(high - low))
    data['tr1'] = abs(np.log(high - close.shift()))
    data['tr2'] = abs(np.log(low - close.shift()))
    tr = data[['tr0', 'tr1', 'tr2']].max(axis=1)
    size =((accountValue*riskFactor) / wwma(tr, atrLength)) #true_range.rolling(atrLength).sum()/atrLength

    return size
..............................................."""

def gap(df,n):
    return (np.log(df['Adj Close']).diff().abs() * 100) > n

def trendFilter(df):
    emaFast = df['Adj Close'].ewm(span=50, adjust=False).mean()
    emaSlow = df['Adj Close'].ewm(span=150, adjust=False).mean()
    emaSlowest = df['Adj Close'].ewm(span=200, adjust=False).mean()
    return (df['Adj Close'] > emaSlow) & (df['Adj Close'] > emaSlowest) & (emaFast > emaSlow) & (emaFast > emaSlowest)

def rTwo(closes):
    returns = np.log(closes)
    x = np.arange(len(returns))
    slope, _, rvalue, _, stderr = linregress(x, returns)
    return (rvalue ** 2) 

In [ ]:
momentumPeriod = 90
rsPeriod = 253

upTrends = []
rsList = []
yes_momentum = []
vols = []
ts = []

for ticker in tickers:
  try:
    df = pdr.get_data_yahoo(ticker, start_date, end_date)
    df = df[df.Volume != 0]

    momentums = df['Adj Close'].rolling(momentumPeriod).apply(mom, raw=False)[-1]
    #momentums = df['Adj Close'].rolling(momentumPeriod).apply(slope, raw=False)[-1]

    vol = df['Adj Close'].rolling(momentumPeriod).apply(rTwo, raw=False)[-1]

    upYes = trendFilter(df)[-1]
    upTrends.extend([upYes])

    rsYes = RS(df, index_df, rsPeriod)[-1]
    rsList.extend([rsYes])
    
    ts.append(ticker)
    yes_momentum.extend([momentums])
    vols.extend([vol])

    print (f'Ticker: {ticker}; Momentum: {momentums}; Volatility: {vol}; UP_TREND: {upYes}; RelativeStrength: {rsYes}\n')
    
    time.sleep(1)
  except :
    pass

Ticker: AAN; Momentum: 0.2606514203719314; Volatility: 0.40209188962281883; UP_TREND: False; RelativeStrength: 0.6362445197719468

Ticker: AAON; Momentum: 1.7334289444030817; Volatility: 0.8697241381998871; UP_TREND: True; RelativeStrength: 0.3533362585347138

Ticker: AAT; Momentum: 0.295260376549362; Volatility: 0.3696469549455831; UP_TREND: True; RelativeStrength: 1.6210962186705542

Ticker: AAWW; Momentum: 1.5217410001606182; Volatility: 0.7900099535167588; UP_TREND: True; RelativeStrength: 2.946064361832404

Ticker: ABCB; Momentum: 0.025286418707461955; Volatility: 0.023425755497031187; UP_TREND: True; RelativeStrength: 1.029062762676524

Ticker: ABG; Momentum: 0.16341649758231377; Volatility: 0.2583363407813277; UP_TREND: False; RelativeStrength: 0.32146279479433304

Ticker: ABM; Momentum: 0.20511949230754156; Volatility: 0.26892360341117727; UP_TREND: False; RelativeStrength: 0.19934542573457892

Ticker: ABTX; Momentum: 1.2288186239532075; Volatility: 0.7041083761283637; UP_TREND

In [ ]:
hl_hm = pd.DataFrame(list(zip(ts, yes_momentum, vols, rsList, upTrends)), columns=['TICKER', 'MOMENTUM', 'VOLATILITY', "RS", "UP"])
momentumRank = hl_hm['MOMENTUM'].rank(method='max', ascending=True)
hl_hm['OVERALL_RANK'] =  momentumRank 
hl_hm = hl_hm.sort_values(by='OVERALL_RANK', ascending=False)
hl_hm = hl_hm[hl_hm.RS > 1.0]
hl_hm = hl_hm[hl_hm.UP]
hl_hm = hl_hm[hl_hm.VOLATILITY > 0.5]
print(hl_hm.head(30), '\n')

    TICKER  MOMENTUM  VOLATILITY         RS    UP  OVERALL_RANK
188   ENTA  7.414841    0.912327   3.218182  True         596.0
575   VRTV  6.469456    0.854704  19.581852  True         595.0
176   ECHO  6.350233    0.832431   1.927833  True         594.0
398   OPRX  5.825177    0.914830   9.579284  True         593.0
503   SPSC  5.056174    0.875626   1.715187  True         592.0
584   WIRE  4.640553    0.860371   5.296206  True         591.0
42    ARCB  4.415889    0.889398   5.667370  True         590.0
522    TBK  4.049695    0.878660   5.437578  True         589.0
48    ASIX  3.920721    0.916645   5.725924  True         588.0
463    RRC  3.886782    0.689594   8.348165  True         587.0
434   PRFT  3.702480    0.874554   6.879344  True         586.0
172   DSPG  3.640739    0.805840   1.309340  True         585.0
333      M  3.619071    0.828097  10.940818  True         584.0
101   CEIX  3.505503    0.726562  13.517018  True         583.0
102   CELH  3.295259    0.834435   9.897